Import Required Package

In [106]:
from pyspark.sql import SparkSession,functions  as f
from pyspark.sql.window import Window as w

Create SparkSession obj 

In [3]:
sp= SparkSession.builder.appName("Test").getOrCreate()

Read required files in Dataframe

In [26]:
df_stock_companies = sp.read.option("inferSchema","true").option("header","true").csv(".\Jupyter_Projects\Stockcompanies.csv")

In [35]:
df_stock_companies.printSchema()
df_stock_companies.show(5)

root
 |-- Ticker_symbol: string (nullable = true)
 |-- Security: string (nullable = true)
 |-- Sector: string (nullable = true)
 |-- Sub_Industry: string (nullable = true)
 |-- Headquarter: string (nullable = true)

+-------------+-------------------+--------------------+--------------------+--------------------+
|Ticker_symbol|           Security|              Sector|        Sub_Industry|         Headquarter|
+-------------+-------------------+--------------------+--------------------+--------------------+
|          MMM|         3M Company|         Industrials|Industrial Conglo...| St. Paul; Minnesota|
|          ABT|Abbott Laboratories|         Health Care|Health Care Equip...|North Chicago; Il...|
|         ABBV|             AbbVie|         Health Care|     Pharmaceuticals|North Chicago; Il...|
|          ACN|      Accenture plc|Information Techn...|IT Consulting & O...|     Dublin; Ireland|
|         ATVI|Activision Blizzard|Information Techn...|Home Entertainmen...|Santa Monica; 

In [39]:
df_stock_price = sp.read.option("inferSchema","true").option("header","true").csv(".\Jupyter_Projects\StockPrices.csv")

In [99]:
df_stock_price.printSchema()
df_stock_price.orderBy('symbol','date').show(5)
df_stock_price.count()

root
 |-- date: date (nullable = true)
 |-- symbol: string (nullable = true)
 |-- open: double (nullable = true)
 |-- close: double (nullable = true)
 |-- low: double (nullable = true)
 |-- high: double (nullable = true)
 |-- volume: integer (nullable = true)

+----------+------+-----------+-----------+-----------+-----------+-------+
|      date|symbol|       open|      close|        low|       high| volume|
+----------+------+-----------+-----------+-----------+-----------+-------+
|2010-01-04|     A|22.45350429|22.38912804|22.26752504|22.62517954|3815500|
|2010-01-05|     A|22.32474893|22.14592346|22.00286123|22.33190343|4186000|
|2010-01-06|     A|22.06723963|22.06723963|22.00286123|22.17453577|3243700|
|2010-01-07|     A| 22.0171681|22.03862589|21.81688126| 22.0457804|3095100|
|2010-01-08|     A|21.91702432|22.03147353|21.74534979|22.06723963|3733900|
+----------+------+-----------+-----------+-----------+-----------+-------+
only showing top 5 rows



851264

Filter and analyze some data in StockCompanies DF

In [50]:
# Filter and get the count of only those sectore which is equal to Industrials 

df_stock_companies.filter(f.lower(f.col("sector")) == "Industrials".lower()).count()

69

In [66]:
#Get the count of all ticker symbol
df_stock_price.groupBy('symbol').count().alias('Count').orderBy('Count', ascending=True).show()

+------+-----+
|symbol|count|
+------+-----+
|   FTV|  126|
|  WLTW|  251|
|  CSRA|  284|
|   HPE|  304|
|  PYPL|  378|
|   KHC|  378|
|   WRK|  385|
|  QRVO|  504|
|   SYF|  504|
|   CFG|  504|
|  NAVI|  504|
|  ALLE|  786|
|  EVHC|  853|
|   NWS|  892|
|  NWSA|  892|
|   MNK|  894|
|  COTY|  896|
|   ZTS|  987|
|  FBHS| 1008|
|  KORS| 1008|
+------+-----+
only showing top 20 rows



In [73]:
df_stock_companies.groupBy('security').count().where(f.col('count')>1).show()

df_stock_companies.where(f.col('security').like('Under%')).show()

+------------+-----+
|    security|count|
+------------+-----+
|Under Armour|    2|
+------------+-----+

+-------------+------------+--------------------+--------------------+-------------------+
|Ticker_symbol|    Security|              Sector|        Sub_Industry|        Headquarter|
+-------------+------------+--------------------+--------------------+-------------------+
|           UA|Under Armour|Consumer Discreti...|Apparel; Accessor...|Baltimore; Maryland|
|          UAA|Under Armour|Consumer Discreti...|Apparel; Accessor...|Baltimore; Maryland|
+-------------+------------+--------------------+--------------------+-------------------+



Fetch data with start date and end date along with their associated closing price for each symbol

In [157]:
df_new_stock = df_stock_price.groupBy(df_stock_price.symbol.alias('sym'))\
    .agg(    f.min(f.col('date')).alias('Min_Date') ,\
             f.max(f.col('date')).alias('Max_Date'))\
    .orderBy('symbol')

#.withColumn('close', f.min(f.col('close')).over(w.partitionBy('symbol').orderBy('date')))\

In [158]:
df_new_stock = df_stock_price.join(df_new_stock,(df_new_stock.Min_Date==df_stock_price.date) & (df_new_stock.sym==df_stock_price.symbol),'inner')\
        .select('sym','Min_Date','Max_Date',df_stock_price.close.alias('Min_Close'))\
        .orderBy('sym')

In [159]:
df_new_stock = df_stock_price.join(df_new_stock,(df_new_stock.Max_Date==df_stock_price.date) & (df_new_stock.sym==df_stock_price.symbol),'inner')\
        .select('symbol','Min_Date','Max_Date','Min_CLose',df_stock_price.close.alias('Max_Close'))\
        .orderBy('symbol')

In [160]:
df_new_stock.show()

+------+----------+----------+-----------+----------+
|symbol|  Min_Date|  Max_Date|  Min_CLose| Max_Close|
+------+----------+----------+-----------+----------+
|     A|2010-01-04|2016-12-30|22.38912804| 45.560001|
|   AAL|2010-01-04|2016-12-30|       4.77| 46.689999|
|   AAP|2010-01-04|2016-12-30|  40.380001|169.119995|
|  AAPL|2010-01-04|2016-12-30|30.57285686|    115.82|
|  ABBV|2013-01-02|2016-12-30|  35.119999| 62.619999|
|   ABC|2010-01-04|2016-12-30|  26.629999| 78.190002|
|   ABT|2010-01-04|2016-12-30|26.12988449|     38.41|
|   ACN|2010-01-04|2016-12-30|      42.07|117.129997|
|  ADBE|2010-01-04|2016-12-30|      37.09|102.949997|
|   ADI|2010-01-04|2016-12-30|      31.67| 72.620003|
|   ADM|2010-01-04|2016-12-30|  31.469999| 45.650002|
|   ADP|2010-01-04|2016-12-30|37.60316242|102.779999|
|   ADS|2010-01-04|2016-12-30|  65.889999|     228.5|
|  ADSK|2010-01-04|2016-12-30|      25.67| 74.010002|
|   AEE|2010-01-04|2016-12-30|      27.76| 52.459999|
|   AEP|2010-01-04|2016-12-3

In [173]:
df_diff_close = df_new_stock.withColumn('Diffrence',f.round(f.col('Max_Close') - f.col('Min_Close'),2))\
                            .withColumn('Profit_%', f.round((f.col('Max_Close') - f.col('Min_Close'))* 100/f.col('Min_Close'),2))\
                            .orderBy('Profit_%')

In [174]:
df_diff_close.show()

+------+----------+----------+-----------+---------+---------+--------+
|symbol|  Min_Date|  Max_Date|  Min_CLose|Max_Close|Diffrence|Profit_%|
+------+----------+----------+-----------+---------+---------+--------+
|   RIG|2010-01-04|2016-12-30|  86.779999|    14.74|   -72.04|  -83.01|
|   SWN|2010-01-04|2016-12-30|  50.830002|    10.82|   -40.01|  -78.71|
|  FSLR|2010-01-04|2016-12-30| 135.460007|    32.09|  -103.37|  -76.31|
|   CHK|2010-01-04|2016-12-30|26.57521381|     7.02|   -19.56|  -73.58|
|   FCX|2010-01-04|2016-12-30| 41.7299995|    13.19|   -28.54|  -68.39|
|  SPLS|2010-01-04|2016-12-30|       24.5|     9.05|   -15.45|  -63.06|
|   FTR|2010-01-04|2016-12-30|       7.96|     3.38|    -4.58|  -57.54|
|   MOS|2010-01-04|2016-12-30|  61.709999|    29.33|   -32.38|  -52.47|
|   NRG|2010-01-04|2016-12-30|  23.870001|    12.26|   -11.61|  -48.64|
|   KMI|2013-01-02|2016-12-30|  36.529999|20.709999|   -15.82|  -43.31|
|   DVN|2010-01-04|2016-12-30|      76.57|45.669998|    -30.9|  

Top 10 Performer

In [180]:
df_diff_close.orderBy(f.col('Profit_%'), ascending = False).limit(10).show()

+------+----------+----------+-----------+----------+---------+--------+
|symbol|  Min_Date|  Max_Date|  Min_CLose| Max_Close|Diffrence|Profit_%|
+------+----------+----------+-----------+----------+---------+--------+
|  NFLX|2010-01-04|2016-12-30|7.639999857|123.800003|   116.16| 1520.42|
|  REGN|2010-01-04|2016-12-30|  24.629999|367.089996|   342.46| 1390.42|
|  ULTA|2010-01-04|2016-12-30|      18.85|254.940002|   236.09| 1252.47|
|   URI|2010-01-04|2016-12-30|      10.04|105.580002|    95.54|  951.59|
|   ALK|2010-01-04|2016-12-30| 8.71749975| 88.730003|    80.01|  917.84|
|   AAL|2010-01-04|2016-12-30|       4.77| 46.689999|    41.92|  878.83|
|   STZ|2010-01-04|2016-12-30|  16.120001|153.309998|   137.19|  851.05|
|  AVGO|2010-01-04|2016-12-30|  18.860001|176.770004|   157.91|  837.27|
|  CHTR|2010-01-05|2016-12-30|       35.0|287.920013|   252.92|  722.63|
|  ORLY|2010-01-04|2016-12-30|       38.5|278.410004|   239.91|  623.14|
+------+----------+----------+-----------+---------

In [188]:
df_diff_close.agg(f.avg(f.col('Profit_%'))).show()

+-----------------+
|    avg(Profit_%)|
+-----------------+
|146.2535329341318|
+-----------------+

